In [ ]:
# The dataset used is CALTECH_101

# CREATING VECTOR DICTIONARY

In [1]:
import tarfile
import os
import shutil
import pickle
import tensorflow as tf
from tensorflow.keras.models import Model
import glob
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50,decode_predictions, preprocess_input

Using TensorFlow backend.


In [2]:
mytar=tarfile.open("101_ObjectCategories.tar.gz")

In [3]:
mytar.extractall("./objects")

In [4]:
# now I have removed all the classes having images <= 50
for f in os.listdir("objects/101_ObjectCategories/"):
    if len(os.listdir("objects/101_ObjectCategories/"+f))<=50:
        shutil.rmtree("objects/101_ObjectCategories/"+f)
        

In [5]:
# Renaming of data
path="objects/101_ObjectCategories/"

for f in os.listdir(path):
    count=0
    for img in os.listdir(path+f):
        os.rename(path+f+"/"+img,path+f+"/"+f+str(count)+".jpg")
        count+=1

In [6]:
# Moving data to combined data
for f in os.listdir(path):
    count=0
    for img in os.listdir(path+f):
        if count < 52:
            shutil.move(path+f+"/"+img,"./combined data/")
        count+=1


In [7]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [8]:
 # creating new model
new_model=Model(model.input,model.layers[-2].output)

In [9]:
def preprocess_image(img):
    img=image.load_img(img,target_size=(224,224,3))
    img=image.img_to_array(img)
    img=img.reshape((1,224,224,3))
    img=preprocess_input(img)
    return img

def encode_image(img):
    img=preprocess_image(img)
    feature_vector=new_model.predict(img)
    feature_vector=feature_vector.reshape((2048,))
    return feature_vector


In [10]:
path="./combined datab/"
vector_dict={}
for img in glob.glob("./combined data/*.jpg"):
    img=str(img)
    r,l=img.split("\\")
    vector=encode_image(img)
    vector_dict[l]=vector
    
    

In [11]:
with open("vector_dict.pkl","wb") as f:
    pickle.dump(vector_dict,f)